In [1]:
import pandas as pd
import requests, os, json
from pandas import json_normalize

In [2]:
suspects = '../../analyses/kindergartens/suspects.csv'
# redflags = '../../analyses/kindergartens/redflags.csv'

enheter = 'data/enheter_2024-06-11T04-16-20.253290563.csv'
underenheter = 'data/underenheter_2024-06-11T04-22-23.463742057.csv'

In [3]:
# Load the CSV file
df_sus = pd.read_csv(suspects, dtype=str)
# df_rf = pd.read_csv(redflags, dtype=str)

all_ids= pd.concat([df_sus[col] for col in df_sus.columns]).unique()
fd = pd.DataFrame(all_ids, columns=['organization number'])

In [4]:
df_en = pd.read_csv(enheter, dtype=str)

In [5]:
df_un = pd.read_csv(underenheter, dtype=str)

In [6]:
# # Check if both dataframes have the exact same list of columns
# columns_match = df_en.columns.equals(df_un.columns)
# print(f"Do both dataframes have the exact same columns? {columns_match}")

# # Identify if one dataframe has more columns than the other
# df_en_columns = set(df_en.columns)
# df_un_columns = set(df_un.columns)

# if df_en_columns != df_un_columns:
#     extra_columns_df_en = df_en_columns - df_un_columns
#     extra_columns_df_un = df_un_columns - df_en_columns
#     common_columns_df = df_un_columns & df_en_columns
#     print(f"Extra columns in df_un: {extra_columns_df_un}")
#     print(f"Extra columns in df_en: {extra_columns_df_en}")
#     print(f"Common columns in df_un: {common_columns_df}")

# # Checking if dataframes have overlapping 'organisasjonsnummer'
# common = pd.merge(df_en, df_un, on='organisasjonsnummer')
# print(common)

In [7]:
fd

,organization number
0,811616532
1,812288172
2,812448072
3,812573772
4,812610082
...,...
6508,979408439
6509,922850097
6510,991982434
6511,971255838


In [8]:
column_fixes = {
    'registreringsdatoIEnhetsregisteret': 'registreringsdatoenhetsregisteret',
    'registrertIMvaRegisteret': 'registrertIMvaregisteret',
}
column_translations = {
    'aktivitet': 'activity',
    'antallAnsatte': 'number of employees',
    'beliggenhetsadresse.adresse': 'location address',
    'beliggenhetsadresse.kommune': 'location municipality',
    'beliggenhetsadresse.kommunenummer': 'location municipality number',
    'beliggenhetsadresse.land': 'location country',
    'beliggenhetsadresse.landkode': 'location country code',
    'beliggenhetsadresse.postnummer': 'location postal code',
    'beliggenhetsadresse.poststed': 'location post town',
    'datoEierskifte': 'ownership change date',
    'forretningsadresse.adresse': 'business address',
    'forretningsadresse.kommune': 'business municipality',
    'forretningsadresse.kommunenummer': 'business municipality number',
    'forretningsadresse.land': 'business country',
    'forretningsadresse.landkode': 'business country code',
    'forretningsadresse.postnummer': 'business postal code',
    'forretningsadresse.poststed': 'business post town',
    'frivilligMvaRegistrertBeskrivelser': 'voluntary VAT registered descriptions',
    'harRegistrertAntallAnsatte': 'has registered number of employees',
    'hjelpeenhetskode.beskrivelse': 'auxiliary unit code description',
    'hjelpeenhetskode.kode': 'auxiliary unit code',
    'hjemmeside': 'website',
    'institusjonellSektorkode.beskrivelse': 'institutional sector code description',
    'institusjonellSektorkode.kode': 'institutional sector code',
    'konkurs': 'bankrupt',
    'konkursdato': 'bankruptcy date',
    'maalform': 'language form',
    'naeringskode1.beskrivelse': 'industry code 1 description',
    'naeringskode1.kode': 'industry code 1',
    'naeringskode2.beskrivelse': 'industry code 2 description',
    'naeringskode2.kode': 'industry code 2',
    'naeringskode3.beskrivelse': 'industry code 3 description',
    'naeringskode3.kode': 'industry code 3',
    'navn': 'name',
    'nedleggelsesdato': 'closure date',
    'oppstartsdato': 'start date',
    'organisasjonsform.beskrivelse': 'organization form description',
    'organisasjonsform.kode': 'organization form code',
    'organisasjonsnummer': 'organization number',
    'overordnetEnhet': 'parent unit',
    'postadresse.adresse': 'postal address',
    'postadresse.kommune': 'postal municipality',
    'postadresse.kommunenummer': 'postal municipality number',
    'postadresse.land': 'postal country',
    'postadresse.landkode': 'postal country code',
    'postadresse.postnummer': 'postal code',
    'postadresse.poststed': 'postal town',
    'registreringsdatoenhetsregisteret': 'registration date in unit register',
    'registrertIForetaksregisteret': 'registration date of the Unit Register',
    'registrertIFrivillighetsregisteret': 'registered in volunteer register',
    'registrertIMvaregisteret': 'registered in VAT register',
    'registrertIStiftelsesregisteret': 'registered in foundation register',
    'sisteInnsendteAarsregnskap': 'last submitted annual accounts',
    'stiftelsesdato': 'foundation date',
    'tvangsavvikletPgaManglendeSlettingDato': 'compulsory liquidation due to lack of deletion date',
    'tvangsopplostPgaMangelfulltStyreDato': 'compulsory dissolution due to insufficient board date',
    'tvangsopplostPgaManglendeDagligLederDato': 'compulsory dissolution due to lack of general manager date',
    'tvangsopplostPgaManglendeRegnskapDato': 'compulsory dissolution due to lack of accounts date',
    'tvangsopplostPgaManglendeRevisorDato': 'compulsory dissolution due to lack of auditor date',
    'underAvvikling': 'under liquidation',
    'underAvviklingDato': 'under liquidation date',
    'underTvangsavviklingEllerTvangsopplosning': 'under compulsory liquidation or dissolution',
    'vedtektsdato': 'articles of association date',
    'vedtektsfestetFormaal': 'statutory purpose'
}
translation_dict = {
    'Underenhet til næringsdrivende og offentlig forvaltning': 'Subunit of businesses and public administration',
    'Enkeltpersonforetak': 'Sole proprietorship',
    'Aksjeselskap': 'Limited company (AS)',
    'Forening/lag/innretning': 'Association/club/facility',
    'Underenhet til ikke-næringsdrivende': 'Subunit of non-business entities',
    'Eierseksjonssameie': 'Condominium',
    'Utenlandsk enhet': 'Foreign entity',
    'Norskregistrert utenlandsk foretak': 'Norwegian registered foreign company (NUF)',
    'Ansvarlig selskap med delt ansvar': 'General partnership with shared liability (DA)',
    'Borettslag': 'Housing cooperative',
    'Ansvarlig selskap med solidarisk ansvar': 'General partnership with joint liability (ANS)',
    'Konkursbo': 'Bankruptcy estate',
    'Samvirkeforetak': 'Cooperative',
    'Tingsrettslig sameie': 'Legal co-ownership',
    'Stiftelse': 'Foundation',
    'Annen juridisk person': 'Other legal entity',
    'Organisasjonsledd': 'Organizational unit',
    'Selskap med begrenset ansvar': 'Company with limited liability',
    'Den norske kirke': 'The Church of Norway',
    'Verdipapirfond': 'Mutual fund',
    'Kommune': 'Municipality',
    'Allmennaksjeselskap': 'Public limited company (ASA)',
    'Interkommunalt selskap': 'Inter-municipal company',
    'Kommunalt foretak': 'Municipal enterprise',
    'Kommandittselskap': 'Limited partnership (KS)',
    'Partrederi': 'Partnership in maritime trade',
    'Særskilt oppdelt enhet, jf. mval. § 2-2': 'Specially divided unit, according to VAT Act § 2-2',
    'Sparebank': 'Savings bank',
    'Pensjonskasse': 'Pension fund',
    'Annet foretak iflg. særskilt lov': 'Other enterprise according to special law',
    'Andre bo': 'Other estates',
    'Boligbyggelag': 'Housing association',
    'Andre enkeltpersoner som registreres i tilknyttet register': 'Other individuals registered in the associated register',
    'Gjensidig forsikringsselskap': 'Mutual insurance company',
    'Administrativ enhet - offentlig sektor': 'Administrative unit - public sector',
    'Kontorfellesskap': 'Office community',
    'Staten': 'The state',
    'Fylkeskommune': 'County municipality',
    'Tvangsregistrert for MVA': 'Compulsory VAT registration',
    'Statsforetak': 'State enterprise',
    'Fylkeskommunalt foretak': 'County municipal enterprise',
    'Europeisk selskap': 'European company (SE)',
    'Andre ikke-juridiske personer': 'Other non-legal persons'
}




In [9]:
# fix columns
df_en = df_en.rename(columns=column_fixes)
df_en = df_en.rename(columns=column_fixes)

# Concatenate the dataframes
df = pd.concat([df_en, df_un], ignore_index=True)
# print(df)
df = df.rename(columns=column_translations)
df['organization form description (EN)'] = df['organization form description'].map(translation_dict)

In [ ]:
# filter for only suspects
df_sus = pd.merge(fd, df, how='left', on='organization number')

In [ ]:
df_sus.to_csv('info_suspects.csv', index=False)

In [ ]:
def column_statistics(df):
    for column in df.columns:
        print(f"==================================================")
        print(f"Column: {column}")
        unique_values = df[column].nunique()
        
        if unique_values == len(df):
            print("  Unique per row")
        elif pd.api.types.is_numeric_dtype(df[column]):
            mean = df[column].mean()
            std = df[column].std()
            min_val = df[column].min()
            max_val = df[column].max()
            print(f"  Mean: {mean}")
            print(f"  Standard Deviation: {std}")
            print(f"  Min: {min_val}")
            print(f"  Max: {max_val}")
        else:
            value_counts = df[column].value_counts()
            print("  Value counts:")
            print(value_counts)
        print()

In [ ]:
column_statistics(df_sus)

In [ ]:
# # Function to get organization details from organization number
# def get_org_details(orgnr):
#     url = f'https://data.brreg.no/enhetsregisteret/api/enheter/{orgnr}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return {'organisasjonsnummer': orgnr, 'status': '404 - Not Found'}

In [ ]:
# # Function to get organization details from organization number
# def get_org_details(orgnr):
#     url = f'https://data.brreg.no/enhetsregisteret/api/enheter/{orgnr}'
#     filename = f'json_data/{orgnr}.json'
#     error_log_file = f'json_data/{orgnr}_error.log'

#     # Check if the JSON file already exists
#     if os.path.exists(filename):
#         with open(filename, 'r') as file:
#             return json.load(file)
#     # Check if the JSON file already exists
#     if os.path.exists(error_log_file):
#         with open(error_log_file, 'r') as file:
#             return {'organisasjonsnummer': orgnr, 'status': file.read()}

#     try:
#         response = requests.get(url, timeout=10)
#         response.raise_for_status()
#         data = response.json()
#         # Save the JSON data to a file
#         os.makedirs('json_data', exist_ok=True)
#         with open(filename, 'w') as file:
#             json.dump(data, file)
#         return data
#     except requests.exceptions.HTTPError as http_err:
#         error_message = f"HTTP error occurred: {http_err}"
#     except requests.exceptions.ConnectionError as conn_err:
#         error_message = f"Connection error occurred: {conn_err}"
#     except requests.exceptions.Timeout as timeout_err:
#         error_message = f"Timeout error occurred: {timeout_err}"
#     except requests.exceptions.RequestException as req_err:
#         error_message = f"An error occurred: {req_err}"

#     # Log the error to a file
#     with open(error_log_file, 'w') as file:
#         file.write(error_message)

#     return {'organisasjonsnummer': orgnr, 'status': error_message}

In [ ]:
# get_org_details('996920917')